***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [ ]:
# if the following command generates an error, you probably didn't enable
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

# Imports & Setup

In [ ]:
!pip install -q google-cloud-storage==1.43.0

In [ ]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

In [ ]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Feb 20 09:06 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession

# conf = SparkConf().set("spark.ui.port", "4050")
# conf.set("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.2-s_2.12")

spark = SparkSession.builder \
    .appName("anchor_index_spark") \
    .master("yarn") \
    .config('spark.executor.memory', '10g') \
    .config('spark.driver.memory', '24g') \
    .config('spark.executor.memoryOverhead', '2g') \
    .config('spark.driver.memoryOverhead', '2g') \
    .config('spark.driver.maxResultSize', '24g') \
    .config('spark.yarn.am.memory', '4g') \
    .config('spark.yarn.am.memoryOverhead', '2g') \
    .config('spark.executor.memory', '2g') \
    .config('spark.sql.shuffle.partitions', '500') \
    .getOrCreate()
sc = SparkContext.getOrCreate()

In [ ]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'bucket_title'
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name.endswith('.parquet'):
        paths.append(full_path + b.name)

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [ ]:
parquetFile = spark.read.parquet(*paths)
doc_title_pairs = parquetFile.select("title", "id").rdd
n_pages = doc_title_pairs.count()

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [ ]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [ ]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [ ]:
from inverted_index_gcp import InvertedIndex

**YOUR TASK (10 POINTS)**: Use your implementation of `word_count`, `reduce_word_counts`, `calculate_df`, and `partition_postings_and_write` functions from Colab to build an inverted index for all of English Wikipedia in under 2 hours.

A few notes:
1. The number of corpus stopwords below is a bit bigger than the colab version since we are working on the whole corpus and not just on one file.
2. You need to slightly modify your implementation of  `partition_postings_and_write` because the signature of `InvertedIndex.write_a_posting_list` has changed and now includes an additional argument called `bucket_name` for the target bucket. See the module for more details.
3. You are not allowed to change any of the code not coming from Colab.

In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links",
                    "may", "first", "see", "history", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
base_dir = "index_title"
NUM_BUCKETS = 124

def token2bucket_id(token):
    return int(_hash(token),16) % NUM_BUCKETS

def get_tokens(text, doc_id):
    """Support function - performs token filtering per doc 
      Parameters:
    -----------
    text: str
      Text of one document
    id: int
      Document id
    Returns:
    --------
    List of tuples
      A list of (doc_id, filtered_token_list) pairs 
    """
    tokens = [token.group() for token in RE_WORD.finditer(text.lower()) if token != '']
    filtered_tokens = [tok for tok in tokens if (tok not in all_stopwords)]
    return [(doc_id, filtered_tokens)]

def calculate_df(postings):
    ''' Takes a posting list RDD and calculate the df for each token.
    Parameters:
    -----------
      postings: RDD
        An RDD where each element is a (token, posting_list) pair.
    Returns:
    --------
      RDD
        An RDD where each element is a (token, df) pair.
    '''
    # Count the number of documents for each token
    df_rdd = postings.map(lambda x: (x[0], len(x[1])))

    return df_rdd

def reduce_word_counts(unsorted_pl):
    ''' Returns a sorted posting list by wiki_id.
    Parameters:
    -----------
      unsorted_pl: list of tuples
        A list of (wiki_id, tf) tuples
    Returns:
    --------
      list of tuples
        A sorted posting list.
    '''
    sorted_pl = sorted(unsorted_pl, key=lambda x: x[0])  # Sort by wiki_id
    return sorted_pl

def word_count(text, doc_id):
    ''' Count the frequency of each word in `text` (tf) that is not included in
    `all_stopwords` and return entries that will go into our posting lists.
    Parameters:
    -----------
      text: str
        Text of one document
      id: int
        Document id
    Returns:
    --------
      List of tuples
        A list of (token, (doc_id, tf)) pairs
        for example: [("Anarchism", (12, 5)), ...]
    '''
    # Count the frequency of each token
    token_freq = {}
    for token in text:
        token_freq[token] = token_freq.get(token, 0) + 1

    # Create list of tuples in the form of (token, (doc_id, tf))
    result = [(token, (doc_id, freq)) for token, freq in token_freq.items()]

    return result

def get_doc_len(tokens,doc_id):
    """ Count document filtered length for storage in index as well as document vector length for RDD calculations
  Parameters:
  -----------
    id: int
      Document id
    tokens: str
      list of tokens from document
    Returns:
  --------
    List of tuples
      A list of (doc_id, doc_length) pairs
  """
    return doc_id, len(tokens)

def partition_postings_and_write(postings):
    ''' A function that partitions the posting lists into buckets, writes out
    all posting lists in a bucket to disk, and returns the posting locations for
    each bucket. Partitioning should be done through the use of `token2bucket`
    above. Writing to disk should use the function  `write_a_posting_list`, a
    static method implemented in inverted_index_colab.py under the InvertedIndex
    class.
    Parameters:
    -----------
      postings: RDD
        An RDD where each item is a (w, posting_list) pair.
    Returns:
    --------
      RDD
        An RDD where each item is a posting locations dictionary for a bucket. The
        posting locations maintain a list for each word of file locations and
        offsets its posting list was written to. See `write_a_posting_list` for
        more details.
    '''
    # Step 1: Partition the posting lists into buckets
    buckets = postings.map(lambda x: (token2bucket_id(x[0]), (x[0], x[1])))

    # Step 2: Group posting lists by bucket ID
    grouped_buckets = buckets.groupByKey().mapValues(list)

    # Step 3 & 4: Write each bucket's posting lists to disk and collect information about their location
    posting_locations = grouped_buckets.map(lambda x: InvertedIndex.write_a_posting_list(x,base_dir,bucket_name))

    return posting_locations

In [ ]:
# word counts map
doc_tok = doc_title_pairs.flatMap(lambda x: get_tokens(x[0], x[1]))
doc_length = doc_tok.map(lambda x: get_doc_len(x[1], x[0])) #map and not flatMap because it didn't work
doc_length_dict = doc_length.collectAsMap()
word_counts = doc_tok.flatMap(lambda x: word_count(x[1], x[0]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)

# filtering postings and calculate df
w2df = calculate_df(postings)
w2df_dict = w2df.collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write(postings).collect()

In [ ]:
# get title dict for inverted index to quickly prepare results
id2title = doc_title_pairs.map(lambda x: (x[1], x[0])).collectAsMap()

In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix=base_dir):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
        super_posting_locs[k].extend(v)

Putting it all together

In [ ]:
index_name = "index_title"
# Create inverted index instance
inverted = InvertedIndex(base_dir=base_dir,name=index_name, bucket_name=bucket_name)
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
inverted._N = n_pages
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
inverted.docID_to_length = doc_length_dict
inverted.docID_to_title_dict = id2title
# write the global stats out
inverted.write_index()
# upload to gs
index_src = f"{index_name}.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp_title/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://index.pkl [Content-Type=application/octet-stream]...
/ [1 files][ 18.4 MiB/ 18.4 MiB]                                                
Operation completed over 1 objects/18.4 MiB.                                     


In [ ]:
!gsutil ls -lh $index_dst

 18.45 MiB  2024-02-20T12:05:42Z  gs://ex3adar/postings_gcp/index.pkl
TOTAL: 1 objects, 19343013 bytes (18.45 MiB)
